In [1]:
from numpy.random import seed
import random
from time import time
random.seed(1)
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
import tensorflow as tf
from keras.models import Model
import keras
from keras.preprocessing import image as kimage
from keras.applications.imagenet_utils import preprocess_input	
from keras.callbacks import TensorBoard
vcg_model = keras.applications.vgg16.VGG16()
import pickle
import pandas as pd
import numpy as np
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional


Using TensorFlow backend.


In [2]:
vcg_model.layers.pop()
vcg_model = Model(inputs=vcg_model.inputs, outputs=vcg_model.layers[-1].output)

In [3]:
#helper function to load images
def load_image(path):
        img = kimage.load_img(path, target_size=(224,224))
        x = kimage.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        return np.asarray(x)

In [4]:
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Merge, Activation, Flatten
from keras.preprocessing import image, sequence
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential

EMBEDDING_DIM = 200

In [5]:
class cap_model():

    def __init__(self,encoding_pickle_path='/Users/gurditchahal/W266/FlickrStyle_v0.9/Flickr8k_text/encoded_images.p',train_set_path='/Users/gurditchahal/W266/FlickrStyle_v0.9/Flickr8k_text/flickr_8k_train_dataset.txt',delimiter='No Delimiter',maxlen=False):
        if maxlen:
             self.max_cap_len = maxlen
        else:
            self.max_cap_len = None
        self.vocab_size = None
        self.index_word = None
        self.word_index = None
        self.total_samples = None
        self.train_set_path=train_set_path
        self.encoded_images = pickle.load( open( encoding_pickle_path, "rb" ) )
        self.delimiter=delimiter
        self.variable_initializer(self.train_set_path)
      
    
    #get word token and ids
    def variable_initializer(self,train_set_path,delimiter='\t'):
        if self.delimiter!='No Delimiter':
            delimiter=self.delimiter
        df = pd.read_csv(train_set_path, delimiter=delimiter)
        df=df[['image_id','caption','source']]
        ###############3
        #df=df.head(1000)#truncate for quality check
        ################
        nb_samples = df.shape[0]
        iterr = df.iterrows()
        caps = []
        for i in range(nb_samples):
            x = next(iterr)
            caps.append(x[1][1])
        self.total_samples=0
        for text in caps:
            self.total_samples+=len(text.split())-1
        print("Total samples : "+str(self.total_samples))
        
        words = [txt.split() for txt in caps]
        unique = []
        for word in words:
            unique.extend(word)

        unique = list(set(unique))
        unique.sort() #reproducability
        self.vocab_size = len(unique)
        self.word_index = {}
        self.index_word = {}
        for i, word in enumerate(unique):
            self.word_index[word]=i
            self.index_word[i]=word
        if bool(not self.max_cap_len):#check if cap_len is empty
             max_len=0   
             for caption in caps:
                if(len(caption.split()) > max_len):
                    max_len = len(caption.split())
             self.max_cap_len = max_len
        print("Vocabulary size: "+str(self.vocab_size))
        print("Maximum caption length: "+str(self.max_cap_len))
        print("Variables initialization done!")

    #create data generator for batches
    def data_generator(self,train_set_path=None, batch_size = 32,delimiter='\t',multiple_sources=False):
        #added multiple sources parameter where encoding is actually a dictionariy of dictionaries, source of data points to each dictionariy, allowing for multiple image key pairs... since senticap and mscoco overlap
        train_set_path=self.train_set_path
        if self.delimiter!='No Delimiter':
            delimiter=self.delimiter
        partial_caps = []
        next_words = []
        images = []
        print("Generating data...")
        gen_count = 0
        df = pd.read_csv(train_set_path, delimiter=delimiter)
        df=df[['image_id','caption','source']]
        ###########
        #df=df.head(1000) #small sample test
        ###########
        nb_samples = df.shape[0]
        iterr = df.iterrows()
        caps = []
        imgs = []
        srcs = []
        if multiple_sources: #if multiple sources, list out where each image/caption is coming from
            for i in range(nb_samples):
                x = next(iterr)
                caps.append(x[1][1])
                imgs.append(x[1][0])
                srcs.append(x[1][2])
        else:
            for i in range(nb_samples):
                x = next(iterr)
                caps.append(x[1][1])
                imgs.append(x[1][0])
            

        total_count = 0
        while 1:
            image_counter = -1
            for text in caps:
                image_counter+=1
                try: #check for missing values/ignore for now
                    if multiple_sources:
                        current_image = multiple_sources[srcs[image_counter]][imgs[image_counter]] #first key through source to get the right dictionairy then key by image name to get the right vgg features
                    else:
                        current_image = self.encoded_images[imgs[image_counter]]
                    
                    if type(current_image)!=type(None): #in case no vgg features
                        for i in range(len(text.split())-1):
                            total_count+=1
                            partial = [self.word_index[txt] for txt in text.split()[:i+1]]
                            partial_caps.append(partial)
                            next_w = np.zeros(self.vocab_size)
                            next_w[self.word_index[text.split()[i+1]]] = 1
                            next_words.append(next_w)
                            images.append(current_image)
    
                            if total_count>=batch_size:
                                next_words = np.asarray(next_words)
                                images = np.asarray(images)
                               
                                partial_caps = sequence.pad_sequences(partial_caps, maxlen=self.max_cap_len, padding='post',truncating='post')
                                total_count = 0
                                gen_count+=1
                                if gen_count%1000==0: #check once in awhile
                                    print("yielding count: "+str(gen_count))
                                yield [[images, partial_caps], next_words]
                                partial_caps = []
                                next_words = []
                                images = []
                    else:
                        print("no encoding")
                        pass
                except KeyError:
                    print("No encoding found for image #: "+str(image_counter)+","+str(imgs[image_counter]))
                    pass
        
    def load_image(self, path):
        img = kimage.load_img(path, target_size=(224,224))
        x = kimage.img_to_array(img)
        return np.asarray(x)

    
    #instantiates language model
    def create_model(self,mode='base', ret_model = False):
        if mode=='base':
            #base_model = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
            #base_model.trainable=False
            image_model = Sequential()
            #image_model.add(base_model)
            #image_model.add(Flatten())
            image_model.add(Dense(EMBEDDING_DIM, input_dim = 4096, activation='relu'))
    
            image_model.add(RepeatVector(self.max_cap_len))
    
            lang_model = Sequential()
            lang_model.add(Embedding(self.vocab_size, 256, input_length=self.max_cap_len))
            lang_model.add(LSTM(256,return_sequences=True))
            lang_model.add(TimeDistributed(Dense(EMBEDDING_DIM)))
    
            model = Sequential()
            model.add(Merge([image_model, lang_model], mode='concat'))
            model.add(LSTM(512,return_sequences=False))
            model.add(Dense(self.vocab_size))
            model.add(Activation('softmax'))
    
            print("Model created!")
    
            if(ret_model==True):
                return model
            model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=
0.00005), metrics=['accuracy'])
            return model
        
  
        
        elif mode=='bidirectional':
            
                        #base_model = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
            #base_model.trainable=False
            image_model = Sequential()
            #image_model.add(base_model)
            #image_model.add(Flatten())
            image_model.add(Dense(EMBEDDING_DIM, input_dim = 4096, activation='relu'))
    
            image_model.add(RepeatVector(self.max_cap_len))
    
            lang_model = Sequential()
            lang_model.add(Embedding(self.vocab_size, 256, input_length=self.max_cap_len))
            lang_model.add(Bidirectional(LSTM(256,return_sequences=True)))
            lang_model.add(TimeDistributed(Dense(EMBEDDING_DIM)))
    
            model = Sequential()
            model.add(Merge([image_model, lang_model], mode='concat'))
            model.add(Bidirectional(LSTM(512,return_sequences=False)))
            #model.add(LSTM(1000,return_sequences=False))
            model.add(Dropout(.3))
            model.add(Dense(self.vocab_size))
            model.add(Activation('softmax'))
    
            print("Model created!")
    
            if(ret_model==True):
                return model
            #model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.0001), metrics=['accuracy'])
            #model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(), metrics=['accuracy'])
            model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=
0.00005), metrics=['accuracy'])
            return model
        
        
        
        
        elif mode=='stacked':
             #base_model = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
             #base_model.trainable=False
            image_model = Sequential()
             #image_model.add(base_model)
             #image_model.add(Flatten())
            image_model.add(Dense(EMBEDDING_DIM, input_dim = 4096, activation='relu'))

            image_model.add(RepeatVector(self.max_cap_len))

            lang_model = Sequential()
            lang_model.add(Embedding(self.vocab_size, 256, input_length=self.max_cap_len))
            lang_model.add(LSTM(256, return_sequences=True))
            lang_model.add(TimeDistributed(Dense(EMBEDDING_DIM)))

            model = Sequential()
            model.add(Merge([image_model, lang_model], mode='concat'))
            model.add(LSTM(512,return_sequences=True))
            model.add(LSTM(512,return_sequences=False))
            model.add(Dropout(0.4))
            model.add(Dense(self.vocab_size))
            model.add(Activation('softmax'))
             
                 
            if(ret_model==True):
                return model
    
            model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=
0.000005), metrics=['accuracy'])
            return model
        else:
            print('UNABLE TO CREATE MODEL/UNABLE TO RECOGNIZE SUGGESTED ARCHITECTURE')
            return
            
            
            
            
    def get_word(self,index):
        return self.index_word[index]
    
    

In [6]:
import numpy as np
from keras.preprocessing import sequence
import nltk



In [16]:
def process_caption(caption):
	caption_split = caption.split()
	processed_caption = caption_split[1:]
	try:
		end_index = processed_caption.index('<end>')
		processed_caption = processed_caption[:end_index]
	except:
		pass
	return " ".join([word for word in processed_caption])

#gives actual sentence once captions generated
def get_best_caption(captions):
    captions.sort(key = lambda l:l[1])
    best_caption = captions[-1][0]
    return " ".join([cg.index_word[index] for index in best_caption])

def get_all_captions(captions):
    final_captions = []
    captions.sort(key = lambda l:l[1])
    for caption in captions:
        text_caption = " ".join([cg.index_word[index] for index in caption[0]])
        final_captions.append([text_caption, caption[1]])
    return final_captions


#image=encoded.tail(1)['encoding'].values[0] where encoded is a dataframe with the encoding array in one column and title in the other
def generate_captions(model, image, beam_size):
	start = [cg.word_index['<start>']]
	captions = [[start,0.0]]
	while(len(captions[0][0]) < cg.max_cap_len):
		temp_captions = []
		for caption in captions:
			partial_caption = sequence.pad_sequences([caption[0]], maxlen=cg.max_cap_len, padding='post',truncating='post')
            
                #given image and current state of sentence, predict next word
			next_words_pred = model.predict([np.asarray([image]), np.asarray(partial_caption)])[0]
			next_words = np.argsort(next_words_pred)[-beam_size:]
			for word in next_words:
				new_partial_caption, new_partial_caption_prob = caption[0][:], caption[1]
				new_partial_caption.append(word)
				new_partial_caption_prob+=next_words_pred[word]
				temp_captions.append([new_partial_caption,new_partial_caption_prob])
		captions = temp_captions
		captions.sort(key = lambda l:l[1])
		captions = captions[-beam_size:]

	return captions

def test_model(weight, img_name, beam_size = 3):
	encoded_images = pickle.load( open( "encoded_images.p", "rb" ) )
	model = cg.create_model(ret_model = True, )
	model.load_weights(weight)

	image = encoded_images[img_name]
	captions = generate_captions(model, image, beam_size)
	return process_caption(get_best_caption(captions))
	#return [process_caption(caption[0]) for caption in get_all_captions(captions)] 

def bleu_score(hypotheses, references):
	return nltk.translate.bleu_score.corpus_bleu(references, hypotheses)


def test_model_on_images(weight, img_dir, beam_size = 3):
	imgs = []
	captions = {}
	"""
	with open(img_dir, 'rb') as f_images:
		imgs = f_images.read().strip().split('\n')
	"""
	df_test = pd.read_csv(img_dir)
	df_test.columns = ['caption', 'id', 'image_id']
	imgs = df_test['image_id'].unique().tolist()
	imgs = imgs[:4000]    
    
	encoded_images = pickle.load( open( "encoded_images_coco_test.p", "rb" ) )
	model = cg.create_model(mode='stacked', ret_model = True)
	model.load_weights(weight)

	f_pred_caption = open('predicted_captions_stacked_coco.txt', 'w')

	for count, img_name in enumerate(imgs):
		print("Predicting for image: "+str(count))
		image = encoded_images[img_name]
		image_captions = generate_captions(model, image, beam_size)
		best_caption = process_caption(get_best_caption(image_captions))
		captions[img_name] = best_caption
		print(img_name+" : "+str(best_caption))
		f_pred_caption.write(img_name+"\t"+str(best_caption)+"\n")
		f_pred_caption.flush()
	f_pred_caption.close()
	"""
	f_captions = open('Flickr8k_text/Flickr8k.token.txt', 'rb')
	captions_text = f_captions.read().strip().split('\n')
	image_captions_pair = {}
	for row in captions_text:
		row = row.split("\t")
		row[0] = row[0][:len(row[0])-2]
		try:
			image_captions_pair[row[0]].append(row[1])
		except:
			image_captions_pair[row[0]] = [row[1]]
	f_captions.close()
	"""

	hypotheses=[]
	references = []
	for img_name in imgs:
		hypothesis = captions[img_name]
		reference = str(df_test[df_test['image_id'] == img_name]['caption'])
		hypotheses.append(hypothesis)
		references.append(reference)

	return bleu_score(hypotheses, references)

In [8]:
import os
os.getcwd()
os.listdir()

['.ipynb_checkpoints',
 'attention_decoder.py',
 'Baseline_Model.ipynb',
 'Baseline_Model_30k_Food.ipynb',
 'Baseline_Model_AO.ipynb',
 'Baseline_Model_Base.ipynb',
 'Baseline_Model_GC.ipynb',
 'Baseline_Model_Stacked.ipynb',
 'basemodel_factor_56609_58609_lr01_D04_Final.h5',
 'basemodel_factor_56609_58609_lr01_D04_Final.json',
 'basemodel_factor_56609_58609_lr01_D04_Final_weights.h5',
 'basemodel_factor_56609_61609_lr0001_D04_Final.h5',
 'basemodel_factor_56609_61609_lr0001_D04_Final_weights.h5',
 'basemodel_factor_56609_61609_lr01_D04_Final.json',
 'basemodel_factor_food_baby.h5',
 'basemodel_factor_food_baby.json',
 'captions.txt',
 'COCO_train_food_category.csv',
 'encoded_images.p',
 'encoded_images_coco_test.p',
 'encoded_images_final.p',
 'factored_food_baby_improvement_00.hdf5',
 'factored_food_baby_improvement_01.hdf5',
 'factored_food_baby_improvement_02.hdf5',
 'factored_food_baby_improvement_03.hdf5',
 'factored_food_baby_improvement_04.hdf5',
 'factored_food_baby_improveme

In [9]:
master_set_path='preproccessed_final_data.csv'
#'/Users/gurditchahal/W266/master_caps_flikr_coco_senti_yummly_sourced.csv'
multiple_sources=pickle.load(open('encoded_images_final.p','rb'))
#"senticap_flikr_yummly_coco_encoding_image_pairs.p"
#cg=cap_model(train_set_path=train_set_path,delimiter=None)
x=pd.read_csv(master_set_path)
x['count']=x['caption'].apply(lambda x: len(x.split()))   #previously, model was padding to 98 or so based off of 1 outlier example, extreme slowdown

print(x['count'].quantile(.9))
#Out[7]: 18.0
print(x['count'].quantile(.95))
#Out[8]: 20.0
print(x['count'].quantile(.99))
#Out[9]: 24.0
print(x['count'].quantile(.999))
#Out[10]: 36.802000000010594

print(x['count'].quantile(.995))
#Out[11]: 27.0




18.0
22.0
38.0
95.98400000002584
49.0


In [10]:
import string
import re
translator = str.maketrans('', '', string.punctuation)
maxlen=25
print(x['source'].value_counts())
reddit=x[x['source']=='reddit'].head(30000)
print(reddit.head())
x2=x[(x['source']=='mscocoval') | (x['source']=='senticap') | (x['source']=='yummly') ]
#x2=x[x['source']=='mscocoval'].head(1000)
x2=pd.concat([x2,reddit],ignore_index=True)
x2['caption']=x2['caption'].apply(lambda sent:' '.join( [w for w in sent.split() if len(w)>1] ))
x2['caption']=x2['caption'].apply(lambda cap: cap.translate(translator))
x2['caption']=x2['caption'].apply(lambda s:re.sub(r'[0-9]+',' <dg> ',s))
#https://stackoverflow.com/questions/50297777/remove-low-frequency-words

from collections import Counter
from itertools import chain

# split words into lists
v = x2['caption'].str.split().tolist() # [s.split() for s in df['Col2'].tolist()]
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
x2['caption'] = [' '.join([j for j in i if c[j] > 3]) for i in v]

#https://stackoverflow.com/questions/44114463/stratified-sampling-in-pandas
#x2=x2.groupby('source', group_keys=False).apply(lambda s: s.sample(min(len(s), 11000),random_state=42))
#x2=pd.concat([x2,reddit],ignore_index=True)
print('\n')
print(x2['source'].value_counts())
print(x2.head())
x2['caption']=x2['caption'].apply(lambda x:x.replace(' end',''))
x2['caption']=x2['caption'].apply(lambda x:'<start> ' + ' '.join(x.split()[1:maxlen-1])+' <end>')
x2['count']=x2['caption'].apply(lambda c: len(c.split())) 
print(x2['count'].quantile(.99))
x2.reset_index(drop=True,inplace=True)
print(x2.tail())
#x2=x2.head(1000)
x2.to_csv('food_training_setOG.csv')
train_set_path='food_training_setOG.csv'

reddit       76180
mscocoval    34592
flikr        30000
yummly       27638
senticap     26607
Name: source, dtype: int64
                  image_id                                            caption  \
118837  94a4n1FoodPorn.jpg  <start> i ate italian pizza with prosciutto cr...   
118838  94a16gFoodPorn.jpg  <start> ate a burrito the size of my forearm i...   
118839  949zm7FoodPorn.jpg  <start> international beer day 2018, august 3 ...   
118840  949ze9FoodPorn.jpg              <start> takoyaki at a ramen bar <end>   
118841  949w4pFoodPorn.jpg  <start> they finally did it and they taste so ...   

        source  count  
118837  reddit     16  
118838  reddit     12  
118839  reddit     14  
118840  reddit      7  
118841  reddit     11  


mscocoval    34592
reddit       30000
yummly       27638
senticap     26607
Name: source, dtype: int64
                        image_id  \
0  COCO_val2014_000000389081.jpg   
1  COCO_val2014_000000389081.jpg   
2  COCO_val2014_000000389081.jpg  

In [11]:
x2[x2['source']=='reddit'].head()

,image_id,caption,source,count
88837,94a4n1FoodPorn.jpg,<start> ate italian pizza with prosciutto crud...,reddit,15
88838,94a16gFoodPorn.jpg,<start> ate burrito the size of my in socal <end>,reddit,10
88839,949zm7FoodPorn.jpg,<start> international beer day <dg> august bee...,reddit,12
88840,949ze9FoodPorn.jpg,<start> takoyaki at ramen bar <end>,reddit,6
88841,949w4pFoodPorn.jpg,<start> they finally did it and they taste so ...,reddit,11


In [12]:
cg = cap_model(train_set_path=train_set_path,encoding_pickle_path='encoded_images_final.p',delimiter=None,maxlen=maxlen)
model = cg.create_model('stacked')
model.load_weights('runMSRPROP_weights_improvement_stacked_04.hdf5')
model.summary() 

Total samples : 1033999
Vocabulary size: 8257
Maximum caption length: 25
Variables initialization done!


C:\Users\AOlson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:228: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 25, 400)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 25, 512)           1869824   
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 8257)              4235841   
_________________________________________________________________
activation_1 (Activation)    (None, 8257)              0         
Total params: 11,714,769
Trainable params: 11,714,769
Non-trainable params: 0
________________________________________________________________

epochs=5
counter = 0
batch_size=100
file_name = 'runMSRPROP_weights_improvement_stacked_{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min')
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
callbacks_list = [checkpoint, tensorboard]
model.fit_generator(cg.data_generator(batch_size=batch_size,multiple_sources=multiple_sources), steps_per_epoch=cg.total_samples/batch_size, epochs=epochs, verbose=0, callbacks=callbacks_list)
#sgd + batch size 100

In [13]:
#Trained Data
encoded_images = pickle.load( open( 'encoded_images_final.p', "rb" ) )
test1=encoded_images['flikr']['3338291921_fe7ae0c8f8.jpg']
#test2=encoded_images['senticap'][]
#del encoded_images
x_cap=generate_captions(model,test1,5)
get_all_captions(x_cap)


[['<start> the hot dog out to the water <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> in <end>',
  17.023454300127923],
 ['<start> the hot dog out to the water <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> and <end>',
  17.031828884035349],
 ['<start> the hot dog out to the nice dog <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>',
  17.610526025295258],
 ['<start> the hot dog out to the great dog <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>',
  17.658328123390675],
 ['<start> the hot dog out to the water <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>',
  17.994346465915442]]

In [14]:
#Test Data
encoded_images = pickle.load( open( 'test_reddit_encoded_images.p', "rb" ) )
test1=encoded_images['92wc8mshittyfoodporn.jpg']
x_cap=generate_captions(model,test1,50)
get_best_caption(x_cap)
test1


array([ 0.        ,  1.41450024,  1.87589884, ...,  0.        ,
        0.        ,  0.7755022 ], dtype=float32)

In [17]:
caption_df_path = 'COCO_train_food_category.csv'
weight = 'runMSRPROP_weights_improvement_stacked_04.hdf5'

#print(test_model(weight, test_image))
print(test_model_on_images(weight, caption_df_path, beam_size=3))

C:\Users\AOlson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:228: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Predicting for image: 0
COCO_train2014_000000344065.jpg : looks like this where you have this looks like
Predicting for image: 1
COCO_train2014_000000221187.jpg : 
Predicting for image: 2
COCO_train2014_000000458756.jpg : that looks delicious
Predicting for image: 3
COCO_train2014_000000221194.jpg : looks delicious
Predicting for image: 4
COCO_train2014_000000565264.jpg : those next
Predicting for image: 5
COCO_train2014_000000229401.jpg : looks like some food in my mouth
Predicting for image: 6
COCO_train2014_000000368668.jpg : looks like to me
Predicting for image: 7
COCO_train2014_000000286751.jpg : 
Predicting for image: 8
COCO_train2014_000000262180.jpg : that is the dogs that is that is
Predicting for image: 9
COCO_train2014_000000147492.jpg : this is that
Predicting for image: 10
COCO_train2014_000000311337.jpg : this is the seen next to the inside of the inside
Predicting for image: 11
COCO_train2014_000000131127.jpg : how to be not to the mouth
Predicting for image: 12
COCO_tr

COCO_train2014_000000065965.jpg : those man of hot dog in the day
Predicting for image: 100
COCO_train2014_000000426422.jpg : it looks like man out for the mouth
Predicting for image: 101
COCO_train2014_000000573892.jpg : there are looking food at market
Predicting for image: 102
COCO_train2014_000000385485.jpg : that is some food that looks delicious
Predicting for image: 103
COCO_train2014_000000352724.jpg : what is the
Predicting for image: 104
COCO_train2014_000000147926.jpg : on that picture out to be
Predicting for image: 105
COCO_train2014_000000254431.jpg : looks like the thing is
Predicting for image: 106
COCO_train2014_000000557546.jpg : 
Predicting for image: 107
COCO_train2014_000000131565.jpg : 
Predicting for image: 108
COCO_train2014_000000360948.jpg : 
Predicting for image: 109
COCO_train2014_000000066041.jpg : nice room with some bread
Predicting for image: 110
COCO_train2014_000000336385.jpg : don have the lot of beautiful in the mouth
Predicting for image: 111
COCO_t

COCO_train2014_000000451515.jpg : all that looks like food
Predicting for image: 197
COCO_train2014_000000009148.jpg : that looks like look like
Predicting for image: 198
COCO_train2014_000000205757.jpg : that looks like me
Predicting for image: 199
COCO_train2014_000000058307.jpg : looks like the picture
Predicting for image: 200
COCO_train2014_000000353221.jpg : fries of the fries are next of and cream <dg> <dg> or or
Predicting for image: 201
COCO_train2014_000000566214.jpg : it looks like it looks like it
Predicting for image: 202
COCO_train2014_000000066503.jpg : this is the food of food
Predicting for image: 203
COCO_train2014_000000189390.jpg : what is it looks good
Predicting for image: 204
COCO_train2014_000000484307.jpg : shit
Predicting for image: 205
COCO_train2014_000000000984.jpg : looks like the food next to the lot of bananas
Predicting for image: 206
COCO_train2014_000000574430.jpg : this looks looks like picture
Predicting for image: 207
COCO_train2014_000000099295.jp

COCO_train2014_000000452014.jpg : recipe
Predicting for image: 296
COCO_train2014_000000009651.jpg : 
Predicting for image: 297
COCO_train2014_000000132533.jpg : sauce on the out of lot
Predicting for image: 298
COCO_train2014_000000091576.jpg : it looks like it
Predicting for image: 299
COCO_train2014_000000026041.jpg : that is that
Predicting for image: 300
COCO_train2014_000000132538.jpg : that looks like the thing is
Predicting for image: 301
COCO_train2014_000000476605.jpg : how you have tasty food
Predicting for image: 302
COCO_train2014_000000320957.jpg : looks like that are on the kitchen
Predicting for image: 303
COCO_train2014_000000542146.jpg : it looks like the recipe
Predicting for image: 304
COCO_train2014_000000263620.jpg : on top around the
Predicting for image: 305
COCO_train2014_000000247247.jpg : this is that good
Predicting for image: 306
COCO_train2014_000000230864.jpg : this looks like been
Predicting for image: 307
COCO_train2014_000000058838.jpg : is some fresh 

...

COCO_train2014_000000145680.jpg : 
Predicting for image: 3646
COCO_train2014_000000055569.jpg : looks like this that looks delicious
Predicting for image: 3647
COCO_train2014_000000260373.jpg : man of some food
Predicting for image: 3648
COCO_train2014_000000506136.jpg : this is the picture is it
Predicting for image: 3649
COCO_train2014_000000268569.jpg : 
Predicting for image: 3650
COCO_train2014_000000383263.jpg : this is man that looks like something
Predicting for image: 3651
COCO_train2014_000000440614.jpg : don eat food but are on the plate
Predicting for image: 3652
COCO_train2014_000000375086.jpg : looks like the recipe
Predicting for image: 3653
COCO_train2014_000000391480.jpg : this looks good
Predicting for image: 3654
COCO_train2014_000000432448.jpg : 
Predicting for image: 3655
COCO_train2014_000000440641.jpg : that is looking
Predicting for image: 3656
COCO_train2014_000000072007.jpg : why
Predicting for image: 3657
COCO_train2014_000000358730.jpg : was they you me
Predi

COCO_train2014_000000580344.jpg : this looks delicious
Predicting for image: 3744
COCO_train2014_000000162551.jpg : 
Predicting for image: 3745
COCO_train2014_000000400124.jpg : that looks delicious
Predicting for image: 3746
COCO_train2014_000000326408.jpg : 
Predicting for image: 3747
COCO_train2014_000000154382.jpg : 
Predicting for image: 3748
COCO_train2014_000000563985.jpg : that looks delicious
Predicting for image: 3749
COCO_train2014_000000490264.jpg : it looks like this sauce
Predicting for image: 3750
COCO_train2014_000000006943.jpg : this is the picture of some cake
Predicting for image: 3751
COCO_train2014_000000359201.jpg : that looks like me
Predicting for image: 3752
COCO_train2014_000000219943.jpg : that looks delicious
Predicting for image: 3753
COCO_train2014_000000334637.jpg : that looks delicious
Predicting for image: 3754
COCO_train2014_000000244528.jpg : that looks delicious
Predicting for image: 3755
COCO_train2014_000000080691.jpg : 
Predicting for image: 3756


COCO_train2014_000000376144.jpg : looks like it
Predicting for image: 3843
COCO_train2014_000000277848.jpg : next to the slices of foods
Predicting for image: 3844
COCO_train2014_000000531812.jpg : that looks delicious
Predicting for image: 3845
COCO_train2014_000000253290.jpg : this looks delicious
Predicting for image: 3846
COCO_train2014_000000171376.jpg : this is it
Predicting for image: 3847
COCO_train2014_000000408946.jpg : why is to be on the front of the plate
Predicting for image: 3848
COCO_train2014_000000507252.jpg : 
Predicting for image: 3849
COCO_train2014_000000163197.jpg : man
Predicting for image: 3850
COCO_train2014_000000081283.jpg : that looks delicious
Predicting for image: 3851
COCO_train2014_000000220548.jpg : this is it
Predicting for image: 3852
COCO_train2014_000000114060.jpg : it looks like it looks like it
Predicting for image: 3853
COCO_train2014_000000540059.jpg : man that that looks is on them
Predicting for image: 3854
COCO_train2014_000000171429.jpg : t

COCO_train2014_000000335752.jpg : that is what the recipe
Predicting for image: 3943
COCO_train2014_000000294795.jpg : this looks like this
Predicting for image: 3944
COCO_train2014_000000016292.jpg : 
Predicting for image: 3945
COCO_train2014_000000368548.jpg : that looks like the sauce
Predicting for image: 3946
COCO_train2014_000000532397.jpg : that looks delicious
Predicting for image: 3947
COCO_train2014_000000548786.jpg : that looks like the kind of meal that is it
Predicting for image: 3948
COCO_train2014_000000229302.jpg : this is that looks delicious
Predicting for image: 3949
COCO_train2014_000000425912.jpg : how this the cake
Predicting for image: 3950
COCO_train2014_000000442298.jpg : 
Predicting for image: 3951
COCO_train2014_000000221120.jpg : 
Predicting for image: 3952
COCO_train2014_000000384961.jpg : man looks like the delicious
Predicting for image: 3953
COCO_train2014_000000368594.jpg : this is man with food
Predicting for image: 3954
COCO_train2014_000000016342.jpg

C:\Users\AOlson\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\AOlson\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\AOlson\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

In [83]:
model.save('master_model.h5')

In [106]:
from keras.models import load_model

m=load_model('testing_weights_improvement_00.hdf5')

/home/g_s_chahal/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:1231: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


ValueError: You are trying to load a weight file containing 6 layers into a model with 2 layers.

In [126]:
import string
translator = str.maketrans('', '', string.punctuation)
r="!example..!!! haha."
print(r.translate(translator))


example haha
